# 🎬 YT Short Clipper - Colab Edition

This notebook allows you to run **YT Short Clipper** directly in Google Colab using a web interface (Gradio).

**Features:**
- 🎥 **Auto Download** YouTube videos
- 🧠 **AI Highlights** (GPT-4) to find best moments
- 📱 **Portrait Crop** (9:16) with face tracking
- ⚡ **Auto Captions & Hooks**

---
**How to use:**
1. Run the **Setup** cell below (Wait for installation to finish)
2. Run the **Start App** cell to launch the GUI
3. Click the **Public URL** (e.g., `https://xxxx.gradio.live`) to open the app

In [ ]:
# @title 1. Setup & Install Dependencies
# @markdown Click the play button to install System & Python dependencies.

import os
import sys
import subprocess

# 1. Clone repository if we are in a fresh Colab instance
# (Check if clipper_core.py exists, if not clone)
if not os.path.exists('clipper_core.py'):
    print("📦 Cloning repository...")
    !git clone https://github.com/jipraks/yt-short-clipper.git temp_repo
    !mv temp_repo/* .
    !rm -rf temp_repo
    print("✅ Repository cloned!")
else:
    print("✅ Repository requirements check passed.")

# 2. Install FFmpeg (Required for video processing)
print("🛠️ Installing FFmpeg...")
!apt-get update -qq && apt-get install -y ffmpeg

# 3. Install Python requirements
print("📥 Installing Python libraries...")
!pip install -r requirements.txt
!pip install gradio

print("\n🎉 Setup Complete! You can now run the app below.")

In [ ]:
# @title 2. Start Clipper App
# @markdown Enter your OpenAI API Key when prompted in the UI.

import gradio as gr
import glob
from openai import OpenAI
from pathlib import Path
from clipper_core import AutoClipperCore

# Ensure output directory exists
COLAB_OUTPUT_DIR = "colab_outputs"
Path(COLAB_OUTPUT_DIR).mkdir(exist_ok=True)

def run_process_video(api_key, video_url, num_clips, sub_lang, model_choice):
    if not api_key or not api_key.strip():
        return None, "❌ Error: Please enter your OpenAI API Key."
    
    # Initialize Client
    try:
        client = OpenAI(api_key=api_key)
        # Simple auth check
        client.models.list()
    except Exception as e:
        return None, f"❌ API Key Error: {str(e)}"

    # Progress Logger
    logs = []
    def log_callback(msg):
        logs.append(msg)
        # Also print to notebook console
        print(msg)

    clipper = AutoClipperCore(
        client=client,
        output_dir=COLAB_OUTPUT_DIR,
        model=model_choice,
        subtitle_language=sub_lang,
        log_callback=log_callback
    )

    try:
        yield None, "⏳ Processing started... check logs below."
        
        # Run Processing
        # Note: In Gradio we can't easily stream logs line-by-line in this simple setup,
        # but we return final logs. For real-time, we'd need a generator.
        clipper.process(video_url, num_clips=int(num_clips))
        
        # Find the latest output folder
        all_dirs = sorted(glob.glob(f"{COLAB_OUTPUT_DIR}/*"), key=os.path.getmtime)
        if not all_dirs:
            yield None, "❌ Error: No output directory found."
            return
            
        latest_dir = all_dirs[-1]
        
        # Get all master.mp4 files recursively
        clips = list(Path(latest_dir).rglob("master.mp4"))
        
        if not clips:
            # Fallback to any mp4 if strict naming fails
            clips = list(Path(latest_dir).glob("*.mp4"))
        
        log_text = "\n".join(logs)
        yield [str(c) for c in clips], f"✅ Success! Processed {len(clips)} clips.\n\nLogs:\n{log_text}"
        
    except Exception as e:
        import traceback
        trace = traceback.format_exc()
        yield None, f"❌ Error Occurred:\n{str(e)}\n\nFull Traceback:\n{trace}\n\nLogs so far:\n" + "\n".join(logs)

# --- Gradio Interface ---
with gr.Blocks(title="YT Short Clipper", theme=gr.themes.Soft()) as app:
    gr.Markdown("# 🎬 YT Short Clipper Web")
    
    with gr.Row():
        with gr.Column(scale=1):
            api_key_input = gr.Textbox(label="OpenAI API Key", type="password", placeholder="sk-proj-...")
            url_input = gr.Textbox(label="YouTube URL", placeholder="https://youtu.be/...")
            
            with gr.Row():
                num_clips_slider = gr.Slider(minimum=1, maximum=10, value=3, step=1, label="Clips Count")
                lang_dropdown = gr.Dropdown(choices=["id", "en", "es", "ja"], value="id", label="Subtitle Language")
            
            model_dropdown = gr.Dropdown(
                choices=["gpt-4o", "gpt-4-turbo", "gpt-3.5-turbo"], 
                value="gpt-4o", 
                label="AI Model"
            )
            
            process_btn = gr.Button("🚀 Start Processing", variant="primary", size="lg")
        
        with gr.Column(scale=1):
            output_gallery = gr.Gallery(label="Generated Clips", columns=2, height="auto")
            status_output = gr.Textbox(label="Status & Logs", lines=15, max_lines=20)

    process_btn.click(
        run_process_video, 
        inputs=[api_key_input, url_input, num_clips_slider, lang_dropdown, model_dropdown], 
        outputs=[output_gallery, status_output]
    )

print("Starting Gradio...")
app.queue().launch(share=True, debug=True)